In [ ]:
from matplotlib import cm
from matplotlib import pyplot
from numpy import linalg
from numpy import random
from random import sample
from scipy import signal 
from scipy import stats
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.metrics.cluster import normalized_mutual_info_score
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict

import dmr
import datetime
import foursquare
import geopandas
import math
import numpy
import pandas
import pickle
import pymongo
import time
import seaborn

In [ ]:
sq_leaf = (5.0, 3.0)

In [ ]:
def leaf(name):
    fig.savefig('lab_plot/'+name+'.eps', dpi=150, format='eps', transparent=True, pad_inches=0.10)

In [ ]:
# TODO maglia fine ds98

In [ ]:
tessellation_uri = 'http://dati.comune.milano.it/dataset/806829b9-134b-40cf-b0e0-03e66c4f76d7/resource/3e355dd1-a8b8-483d-ac4d-03a62232ef38/download/ace_maggio_2011.geojson'

# Lantent Activity in Mobility Patterns

In [ ]:
categories_name = ["Cultura",
 "Istruzione",
 "Eventi",
 "Cibi",
 "Lavoro",
 "Night",
 "Ricreativi",
 "Negozi",
 "Trasporti",
 "Residenza"]

In [ ]:
# categories_name = ["Arte e intrattenimento",
#  "Istituti superiori e università",
#  "Eventi",
#  "Cibi",
#  "Posti professionali e altri",
#  "Locali notturni",
#  "All'aperto & Ricreativi",
#  "Negozi e servizi",
#  "Viaggi e trasporti",
#  "Residenza"]

### Pattern di mobilità

In [ ]:
bikemi_dataframe = pandas.DataFrame()

In [ ]:
station_lookup = {}
fwd_lookup, bwd_lookup = {} , {}

In [ ]:
def update_bwd():
    global bwd_lookup
    bwd_lookup = { v:k for k,v in fwd_lookup.items()}

In [ ]:
columns_filter = [ 'Cliente'
                  , 'Data_prelievo'
                  , 'Gio_settimana_prelievo'
                  , 'Festivo_feriale_prelievo'
                  , 'Stazione_prelievo'
                  , 'Durata_sec'
                  , 'Data_arrivo'
                  , 'Gio_settimana_arrivo'
                  , 'Festivo_feriale_arrivo'
                  , 'Stazione_arrivo'
                 ]

In [ ]:
# # unione dei dataframe
# for yy in range(2015, 2019): 
#     for mm in range(1, 13):
#         try:
#             csv_uri = '/home/datasets/bikemi/{0}/{1:02d} {0}.csv'.format(yy,mm)
#             next_df = pandas.read_csv(csv_uri,
#                            lineterminator ='\r',
#                            encoding = 'iso8859_2',
#                            sep = ';',
#                            parse_dates = ['Data_prelievo','Data_arrivo'],
#                            date_parser = lambda x: datetime.datetime.strptime(x,'%d/%m/%y %H:%M'),
#                            decimal = '.'
#                           )
            
#         except FileNotFoundError:
#             pass
#         else:
#             print(yy,mm,end='\r')
#             for _, i in next_df.iterrows():
                
# #                 # LOOKUP
# #                 a = i['Stazione_prelievo']
# #                 b = i['Stazione_arrivo']
                
# #                 if a not in station_lookup:
# #                     station_lookup[a] = {i['Nome_stazione_prelievo']}
# #                 else:
# #                     station_lookup[a].add(i['Nome_stazione_prelievo'])
                    
# #                 if b not in station_lookup:
# #                     station_lookup[b] = {i['Nome_stazione_arrivo']}
# #                 else:
# #                     station_lookup[b].add(i['Nome_stazione_arrivo'])
#                 pass
                
#             #SAVE
#             bikemi_dataframe = pandas.concat([bikemi_dataframe, next_df[columns_filter]])


# pickle.dump(bikemi_dataframe, open( "archive/bikemi_dataframe.pkl", "wb" ) )
# # pickle.dump(station_lookup, open( "archive/station_lookup.pkl", "wb" ) )

In [ ]:
bikemi_dataframe = pickle.load(open('archive/bikemi_dataframe.pkl', 'rb'))

In [ ]:
tmp = pickle.load(open('archive/station_lookup.pkl', 'rb'))
station_lookup = {k:list(v)[0] for k,v in tmp.items()}

In [ ]:
gps = pickle.load(open('/home/datasets/bikemi/station_gps_location.pkl', 'rb'))

### Correzione stazioni 

In [ ]:
# stazioni inutilizzate
for i in list(gps):
    if i not in station_lookup:
        del gps[i]

In [ ]:
# stazioni mancanti
gps[2] = gps[402]
gps[903] = gps[263]

gps[90] = (45.484649, 9.195576)
gps[92] = (45.465589, 9.186123)

In [ ]:
# relookupping
new_gps = []
for i in range(max(gps)+1):
    if i in gps:
        fwd_lookup[i] = len(new_gps)
        new_gps.append(gps[i])
gps = new_gps
update_bwd()

### Correzione POIs 

In [ ]:
# 4square
CLIENT_ID = 'XDNRQIG15QP4PUZEMAGGVBPYHV1WXQMFXXAXZM410USDSTC3'
CLIENT_SECRET = 'M55BW0UXPLUGACETI5ENMSF3WXGLDIENZAHE5VXPPVBYLSWO'
client = foursquare.Foursquare(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
categories = client.venues.categories()

In [ ]:
# TODO rimozione delle 10 categorie madri

In [ ]:
# lookup macrocateg
category_lookup = {}

In [ ]:
def extract_category(c):
    if not c['categories']:
        return [(c['id'],c['name'])]
    else:
        l = [(c['id'],c['name'])]
        for ca in c['categories']:
            l.extend(extract_category(ca))
        return l

In [ ]:
for macro_category in categories['categories']:
    for id_cat, name_cat in extract_category(macro_category):
        category_lookup[id_cat] = (macro_category['id'],macro_category['name'],name_cat)

In [ ]:
# 4square DB
mongo_conn = pymongo.MongoClient("mongodb://marvin.nptlab.di.unimi.it")
six_db = mongo_conn['FoursquarePlacesMilan']
places = six_db['places']

In [ ]:
pois = []

In [ ]:
# pulizia POIS
for i in places.find():
        tmp = {
            'name':i['name'],
            'location':(
                float(i['location']['lat']['$numberDouble']),
                float(i['location']['lng']['$numberDouble'])
            ),
            'categories':{category_lookup[j['id']][0] for j in i['categories']}
        }
        pois.append(tmp)

### Tassellamento 

In [ ]:
raw_aces = geopandas.read_file(tessellation_uri)
vor = {}

In [ ]:
stats.variation([i for i in raw_aces.geometry.area])

In [ ]:
y, x = [pois[i]['location'][0] for i in range(len(pois))], [pois[i]['location'][1] for i in range(len(pois))]

In [ ]:
plot = raw_aces.plot(figsize=sq_leaf, color='dimgray', edgecolor='white', alpha=1.0)
fig = plot.get_figure()
ax = fig.gca()
ax.plot(x, y, marker='.',ms=0.2, linestyle='', color='orangered')
ax.set_xlim(9.06, 9.28)
ax.set_ylim(45.39, 45.54)

In [ ]:
leaf('pois-galaxy')

In [ ]:
# poligoni delle regioni
for _, r in raw_aces.iterrows():
    lat = r.geometry.exterior.xy[1]
    lng = r.geometry.exterior.xy[0]
    vor[int(r.ACE)-1] = Polygon([(lat[i], lng[i]) for i in range(len(lat))]) # L'ARRAY ACE PARTE DA 1

vor = numpy.array([vor[i] for i in range(len(vor))])

In [ ]:
# relookupping
for g in range(len(gps)):
    for r in range(len(vor)):
        if vor[r].contains(Point(gps[g])):
            fwd_lookup[bwd_lookup[g]] = r

In [ ]:
tmp = [float(i+1) for i in list(set(fwd_lookup.values()))]
qqq = raw_aces[raw_aces.ACE.isin(tmp)]

In [ ]:
plot = qqq.plot(figsize=sq_leaf, color='dimgray', edgecolor='white', alpha=1.0)
fig = plot.get_figure()
ax = fig.gca()
ax.set_xlim(9.085, 9.24)
ax.set_ylim(45.42, 45.54)
x, y = [gps[i][1] for i in range(len(gps))], [gps[i][0] for i in range(len(gps))]
ax.plot(x, y, marker='.',ms=3.75, linestyle='', color='gold')

In [ ]:
leaf('bikemi-galaxy')

In [ ]:
# regioni non vuote
unempty = numpy.array([True]*len(vor))
for r in range(len(vor)):
    unempty[r] = r in fwd_lookup.values()

vor = vor[unempty]
# vor = vor[list(set(fwd_lookup.values()))]

In [ ]:
# relookupping
for k,v in dict(zip((i for i, kept in enumerate(unempty) if kept), range(len(vor)))).items():
    for f,b in fwd_lookup.items():
        if b == k:
            fwd_lookup[f] = v

### TF-IDF e SVD

In [ ]:
# categorie: solo quelle usate, senza lista completa
categories = set()
for i in pois:
    for j in i['categories']: 
        categories.add(j)
categories = list(categories)
categories.sort()

#### Frequenze assolute

In [ ]:
# abs_frequencies = numpy.zeros([len(vor), len(categories)])
# for r in range(len(vor)):
#     for p in pois:
#         if vor[r].contains(Point(p['location'])):
#             for k in p['categories']:
#                 abs_frequencies[r, categories.index(k)] += 1
# pickle.dump(abs_frequencies, open( "archive/abs_frequencies.pkl", "wb" ) )

In [ ]:
# abs_frequencies = numpy.zeros([len(vor), len(categories)])
# for r in range(len(vor)):
#     for p in pois:
#         if vor[r].contains(Point(p['location'])):
#             for k in p['categories']:
#                 abs_frequencies[r, random.choice(len(categories))] += 1

In [ ]:
abs_frequencies = pickle.load(open('archive/abs_frequencies.pkl', 'rb'))

In [ ]:
# abs_frequencies = numpy.delete(abs_frequencies, 2, 1)
# del categories[2]
# del categories_name[2]

In [ ]:
dim_cat = [sum(abs_frequencies[:,k]) for k in range(len(categories))] # n° poi per categoria
dim_reg = [sum(abs_frequencies[r,:]) for r in range(len(vor))] # n°  poi per regione
num_reg = [len(abs_frequencies[:,k][abs_frequencies[:,k] != 0]) for k in range(len(categories))] # n° regioni per categoria

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ax.barh(categories_name, dim_cat, color='darkgreen', log=True)

In [ ]:
leaf('category-hist')

In [ ]:
tf_idf = numpy.zeros([len(vor),len(categories)])

In [ ]:
for i in range(len(vor)):
    for j in range(len(categories)):
        tf = dim_cat[j]/dim_reg[i]
        idf = len(vor)/num_reg[j]
        tf_idf[i,j] = tf*1 # math.log(idf)

In [ ]:
u, s, v = linalg.svd(tf_idf, full_matrices=True)

In [ ]:
# TODO low-rank di s, e U, V ridotte

In [ ]:
# Collaborative features
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ax = seaborn.heatmap(tf_idf, linewidth=0.05)

### Cuboidi

In [ ]:
time_gran = 60*10
time_max = 60*60*18*2
time_bin = [i for i in range(0,time_max,time_gran)]

In [ ]:
def i_time_bin(t,f):
    return int((60*(60*(18*(f)+((t.hour+17)%24))+t.minute)+t.second)/time_gran)

In [ ]:
# def bin_rev_i(i):
#     i *= time_gran
#     l = int(i/(60*60*24*2*len(vor))%(2))
#     r = int(i/(60*60*24*2)%(len(vor)))
#     f = int(i/(60*60*24)%(2))
#     h = int(i/(60*60)%(24))
#     m = int(i/(60)%(60))
#     return {'arriving_or_leaving':l, 'region':r, 'work_or_holy':f, 'hour':h, 'min':m}

In [ ]:
arriving_cuboid = numpy.zeros([len(vor), len(vor), len(time_bin)])
leaving_cuboid = numpy.zeros([len(vor), len(vor), len(time_bin)])

In [ ]:
arr_users = numpy.array([[[set()]*len(time_bin)]*len(vor)]*len(vor))
leav_users = numpy.array([[[set()]*len(time_bin)]*len(vor)]*len(vor))

In [ ]:
# for i, r in bikemi_dataframe.iterrows():
#     src = fwd_lookup[r['Stazione_prelievo']]
#     dst = fwd_lookup[r['Stazione_arrivo']]
#     intr = range(1,7)
    
#     if(r['Data_arrivo'].hour not in intr and r['Data_prelievo'].hour not in intr):
        
#         arr_time = i_time_bin(r['Data_arrivo'], r['Festivo_feriale_arrivo'])
#         leav_time = i_time_bin(r['Data_prelievo'], r['Festivo_feriale_prelievo'])
#         usr = r['Cliente']
        
#         if usr not in arr_users[src, dst, arr_time]:
#             arriving_cuboid[src, dst, arr_time] += 1
#             arr_users[src, dst, arr_time].add(usr)
#         if usr not in leav_users[src, dst, leav_time]:
#             leaving_cuboid[src, dst, leav_time] += 1
#             leav_users[src, dst, arr_time].add(usr)
        
# pickle.dump(arriving_cuboid, open( "archive/arriving_cuboid.pkl", "wb"))
# pickle.dump(leaving_cuboid, open( "archive/leaving_cuboid.pkl", "wb"))

In [ ]:
arriving_cuboid = pickle.load(open('archive/arriving_cuboid.pkl', 'rb'))
leaving_cuboid = pickle.load(open('archive/leaving_cuboid.pkl', 'rb'))

In [ ]:
docs = {}

In [ ]:
for r in range(len(vor)):
    a = arriving_cuboid[:,r,:].ravel()
    l = leaving_cuboid[r,:,:].ravel()
    m = a + l
    v = u[r]+[1]
    docs[r] = {'v':v, 'm':m}

In [ ]:
vocab = set(numpy.array([docs[r]['m'] for r in range(len(vor))]).ravel())
vocab = list(vocab)
vocab.sort()

In [ ]:
tmp = []
for r in range(len(vor)):
    tmp += docs[r]['m'].tolist()

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
eps = 1
yell = 1
y, x = numpy.histogram(tmp, numpy.arange(0, max(tmp)+eps, eps))
ax.plot(x[yell:-1], y[yell:], marker='.',ms=5.5, linestyle='', color='maroon')

In [ ]:
#  # remove zeros?
# docs[r]['m'] = docs[r]['m'][ docs[r]['m'] != 0]

In [ ]:
# fig = pyplot.figure(figsize=sq_leaf)
# ax = fig.gca()
# ecdf = ECDF(tmp)
# ax.plot(ecdf.x, ecdf.y, marker='.',ms=0.1, linestyle='--', color='coral')
# ax.set_xlim(0, 40)

In [ ]:
# numpy.mean(tmp), numpy.percentile(tmp, 50), numpy.std(tmp)

###  Probablilità congiunta

In [ ]:
probs = numpy.zeros([len(vor), len(categories)])
for r in range(len(abs_frequencies)):
    probs[r] = abs_frequencies[r]/numpy.sum(abs_frequencies[r])

In [ ]:
# pickle.dump(probs, open("means_k/probs/probs.pkl", "wb"))

In [ ]:
move_probs = numpy.zeros([len(vor),len(vor)])

In [ ]:
# users = numpy.array([[set()]*len(vor)]*len(vor))

In [ ]:
# for i, r in bikemi_dataframe.iterrows():
#     src = fwd_lookup[r['Stazione_prelievo']]
#     dst = fwd_lookup[r['Stazione_arrivo']]
#     usr = r['Cliente']

#     if usr not in users[src, dst]:
#         move_probs[src, dst] += 1
#         users[src, dst].add(usr)
            
# pickle.dump(move_probs, open( "archive/move_probs.pkl", "wb"))

In [ ]:
move_probs = pickle.load(open( "archive/move_probs.pkl", "rb" ))

In [ ]:
conj_probs = numpy.zeros([len(categories),len(categories)])
tmp = numpy.sum(move_probs)
for ks in range(len(categories)):
    for kd in range(len(categories)):
        for rs in range(len(vor)):
            for rd in range(len(vor)):
                conj_probs[ks,kd] += probs[rs,ks]*probs[rd,kd]*move_probs[rs,rd]/tmp

In [ ]:
cond_probs = numpy.zeros([len(categories),len(categories)])
for ks in range(len(categories)):
    tmp = numpy.sum(conj_probs[ks,:])
    for kd in range(len(categories)):
        cond_probs[ks,kd] = conj_probs[ks,kd]/tmp

In [ ]:
# Category ditribution over each region

fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
# ax = seaborn.heatmap(abs_frequencies, linewidth=0.5)
ax = seaborn.heatmap(probs, linewidth=0.05)
# ax = seaborn.heatmap(move_probs, linewidth=0.5)
# ax = seaborn.heatmap(conj_probs, linewidth=0.5)
# ax = seaborn.heatmap(cond_probs, linewidth=0.5)

In [ ]:
{categories_name.index(i):i for i in categories_name}

### Diriclet Multinomial Regression di Kato 

In [ ]:
        '''
            - z_m_n: topics assigned to word slots in documents
            - n_m_z: freq. of topics assigned to documents
            - n_z_w: freq. of words assigned to topics
            - n_z:   freq. of topics assigned
        '''

In [ ]:
corpus, coeff_vecs = [], []

In [ ]:
# params
K = 10
sigma = 0.01
beta = 0.01

In [ ]:
for r in range(len(docs)):
    corpus.append(numpy.array(docs[r]['m']))
    coeff_vecs.append(numpy.array(docs[r]['v']))
corpus, coeff_vecs = numpy.array(corpus), numpy.array(coeff_vecs)

In [ ]:
# learning
voca = dmr.Vocabulary()
doku = voca.read_corpus(corpus)

In [ ]:
lda = dmr.DMR(K, sigma, beta, doku, coeff_vecs, voca.size())

In [ ]:
# pickle.dump(corpus, open( "kato_dmr/corpus.pkl", "wb" ) )
# pickle.dump(coeff_vecs, open( "kato_dmr/coeff_vecs.pkl", "wb" ) )

In [ ]:
lda.learning(iteration=150, voca=voca)

In [ ]:
# lda = pickle.load(open("kato_dmr/55lda.pkl","rb"))

In [ ]:
# # topic probability of each document
# tdist = lda.topicdist()
# extracted = []
# for doc, vec, td in zip(corpus, vecs, tdist):
#     print("For: ", doc, "Max topic: ", np.argmax(td), "Max prob.: ", np.max(td))
#     #print("ALPHA", np.dot(vec, lda.Lambda.T))
#     extracted.append(np.argmax(td))

#### Results 

In [ ]:
th = lda.topicdist() # topic probability of each document
numpy.shape(th)

In [ ]:
b = lda.worddist() # word probability of each topic
numpy.shape(b)

In [ ]:
z = lda.z_m_n # maybe
numpy.shape(z)

#### Region-Topic Heatmap  

In [ ]:
cat_heat = th
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ax = seaborn.heatmap(cat_heat, linewidth=0.5)

#### Assignment Distribution

In [ ]:
y = lda.n_z

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ax.bar(range(len(y)),y)

In [ ]:
reg_a, arr_or_leav, reg_b, work_or_holy, hh, mm = fwd_lookup[162], 0, fwd_lookup[56], 0, 9, 20
i = (24*(2*(50*(arr_or_leav)+reg_b)+work_or_holy)+hh)+int(mm/10)

In [ ]:
# fig.savefig('15iter_topic_assig.png', dpi=600)
# pickle.dump(lda.n_m_z, open( "means_k/kato.pkl", "wb" ) )

### K Means 

In [ ]:
for i in range(len(th)):
    th[i] /= numpy.sum(th[i])

In [ ]:
pickle.dump(probs, open("means_k/probs/probs.pkl", "wb"))
pickle.dump(th, open("means_k/kato/kato.pkl", "wb"))

In [ ]:
# implementate k-means qui è improponibile

In [ ]:
0/0

In [ ]:
pre_clust = pickle.load(open("means_k/probs/probs_avg.pkl","rb"))
post_clust = pickle.load(open("means_k/kato/kato_avg.pkl","rb"))

In [ ]:
# score post-lda
y = [post_clust[1][i] for i in post_clust[1]]
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ax.plot(list(post_clust[1].keys()), y, marker='o',ms=5.5, linestyle='-', color='orange')

### Mutual Score

In [ ]:
normalized_mutual_info_score(pre_clust[0][10],post_clust[0][10])

In [ ]:
a,b = pre_clust[0][10], post_clust[0][10]

In [ ]:
def proper_index(a, b):
    metrics = {}
    for label in set(a):
        metrics[label] = [0.0, 0.0]
        inner_labels = set(b[numpy.where(a == label)])
        for proper_label in inner_labels:
            t_set = set(numpy.where(b == proper_label)[0])
            a_set = set(numpy.where(a == label)[0])
            metrics[label][0] += len(t_set - a_set)
            metrics[label][1] += len(t_set)
        metrics[label] = metrics[label][0] / metrics[label][1]
    return metrics

In [ ]:
tmp=[]
for a in pre_clust[0].values():
    for b in post_clust[0].values():
        tmp.append(list(proper_index(a, b).values()))
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
ax.boxplot(tmp) # pre k = varianza , post k= propritetà

In [ ]:
# leaf(fig,'improprity-distro')

In [ ]:
0/0

In [ ]:
# sample algorthm try
n = 4
x = numpy.array([i for i in range(n) for j in range(n)])
y = numpy.array([i for i in range(int(n/2)) for j in range(n*2)])
proper_index(y,x)

### DRM di Zheng

In [ ]:
zheng_z = pickle.load(open("zheng_dmr/z.pkl","rb"))

In [ ]:
tmp = zheng_z
t = [ tmp[j][i] for j in range(len(tmp)) for i in range(len(tmp[j])) ]

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
y, x = numpy.histogram(t, numpy.arange(0, max(t)+2, 1))
ax.plot(x[:-1], y, marker='.',ms=11.0, linestyle='', color='purple')

### Metrics 

In [ ]:
qt = {}
for i, r in bikemi_dataframe.iterrows():
    t = r.Data_prelievo.date()
    if t not in qt:
        qt[t] = set()
    qt[t].add(r['Cliente'])

In [ ]:
tmp = [len(i) for i in qt.values()]
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
eps = 400
yell = 0
y, x = numpy.histogram(tmp, numpy.arange(0, max(tmp)+eps, eps))
ax.plot(x[yell:-1], y[yell:], marker='.',ms=5.5, linestyle='-', color='maroon')

In [ ]:
fig = pyplot.figure(figsize=sq_leaf)
ax = fig.gca()
from scipy import stats
stats.probplot(tmp, dist="norm", plot=ax)
# pylab.show()

In [ ]:
numpy.mean(tmp), numpy.median(tmp), numpy.percentile(tmp, 25), numpy.percentile(tmp, 75)